integrate the add event code into functions

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as random
import csv
import heapq
from tqdm import tqdm
from math import factorial

In [30]:
# Hyper parameters
EPISODES = 100 # times every setting is replicated
# random.seed(1)
WriteFile = True
FileAdd = '/Users/valarian/SJTU/SJTU/毕业论文/A2M4testPoolCentralSimulation.csv'
WarmTime = 500
RunTime = 30000

In [31]:
# set the parameters
# get the number of states S
'''
A: number of areas
M: number of total bikes
S: number of total states
Pij: transfering possibility matrix
Beta: broken rate
ArrMtx: arrival rates of each area
Gamma: gathering rate
Mu: fix rate
Delta: distributing rate
RhoMtx: matrix of ride rates
N: number of fix servers
B_: valve value at broken pool
D_: valve value at distributing pool
'''

A = 2
M = 4
S = int(factorial(A+2+A**2+M)/factorial(A+2+A**2)/factorial(M))
print(S)
Pij = [[0.3, 0.7],
       [0.7, 0.3]]
Beta = 0.3
ArrLst = [5.0, 5.0]
Gamma = 1.0
Mu = 2.0
Delta = 1.0
RhoMtx = [[1.0, 1.0], 
          [1.0, 1.0]]
N = 2
B_, D_ = 2, 2

495


In [43]:
# simulate the central model
# with BP, RC, DP

class Model():
    '''
    This is the central model
    '''
    # initiate the parameters in this function
    def __init__(self):     
        self.timeLimit = WarmTime + RunTime
        self.areas = list(range(A))
        self.epi = 0
        
    def reset(self):
        self.T = 0 # time cursor
        self.formerT = 0
        self.servedCustomers = 0 # number of customer served during runtime in an episode

        self.state1 = [int(M/A)]*A + [0]*3 # Nis, BP, FC, and DP
        self.state2 = [[0]*A for i in range(A)] # Rijs
        self.F = [] # time to be empty for fixing queue
        heapq.heapify(self.F)
        
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(A):
            heapq.heappush(self.scheduler, [random.expovariate(ArrLst[i]), -1, i, i])
        self.stateRecord = self.state1[:A] + self.state2[0] + self.state2[1] + self.state1[-3:]
        return self.state1, self.state2, self.T
    
    def getRecord(self):
        result = [self.epi]
        result += self.stateRecord
        result.append((self.T - self.formerT)/(WarmTime+RunTime))
        self.formerT = self.T
        self.stateRecord = self.state1[:A] + self.state2[0] + self.state2[1] + self.state1[-3:]
        return result

    def setPerformance(self):
        if self.T>WarmTime: 
            self.servedCustomers += 1
                #print(self.servedCustomers)
    
    def simulate(self):
        if WriteFile:
            with open(FileAdd, 'w') as fout:
                writer = csv.writer(fout)
                for i in range(EPISODES):
                    self.reset()
                    self.epi = i
                    while self.T <= self.timeLimit:
                        # print(self.T)
                        self.stepForward()
                        writer.writerow(self.getRecord())
                        #if self.T > WarmTime:
        else:
            for i in range(EPISODES):
                self.reset()
                while self.T <= self.timeLimit:
                    self.stepForward()

        return self.servedCustomers / (self.T-WarmTime)
                
    def getRidingTime(self, s, t):
        rowS, colS, rowT, colT = s//A, s%A, t//A, t%A
        if s==t: r = 0.5
        else: r = abs(rowS-rowT) + abs(colS-colT)
        return random.expovariate(r)

    def addEvent(self, kind):
        if kind == -1:
            next_time = random.expovariate(ArrLst[self.start]) + self.T
            start, end = self.start, self.start
        elif kind == 1:
            next_time = random.expovariate(RhoMtx[self.start][self.terminal]) + self.T
            start, end = self.start, self.terminal
        elif kind == 2: 
            next_time = random.expovariate(Gamma)
            next_time += self.T
            start, end = 'b', 'f'
            #print('add event 2')
        elif kind == 3:
            next_time = random.expovariate(Mu) 
            if self.state1[-2] < N:
                next_time += self.T 
                heapq.heappush(self.F, next_time)
            else: 
                next_time += heapq.heappop(self.F)
                heapq.heappush(self.F, next_time)
            start, end = 'f', 'd'
        else: 
            next_time = random.expovariate(Delta)
            next_time += self.T
            start, end = 'd', 'ni'
        heapq.heappush(self.scheduler, [next_time, kind, start, end])
        
    def bikeArr(self):
        self.state2[self.start][self.terminal] -= 1
        if random.random()<Beta:
            self.state1[-3] += 1
            heapq.heappop(self.scheduler)
            if self.state1[-3] == B_:
                self.addEvent(2)
        else:
            self.state1[self.terminal] += 1
            heapq.heappop(self.scheduler)
    def BPover(self):
        heapq.heappop(self.scheduler)
        for i in range(B_): 
            self.state1[-3] -= 1
            self.addEvent(3) 
            self.state1[-2] += 1
        if self.state1[-3] >= B_: self.addEvent(2)
    def repair(self):
        heapq.heappop(self.scheduler)
        if self.state1[-2] <= N: heapq.heappop(self.F)
        self.state1[-2] -= 1
        self.state1[-1] += 1
        if self.state1[-1] == D_:
            self.addEvent(4)
    def DPover(self):
        heapq.heappop(self.scheduler)
        self.state1[-1] -= D_
        for i in range(A): self.state1[i] += D_/A
        if self.state1[-1] >= D_: self.addEvent(4)
    def cusArr(self):
        #print(self.state1, self.state2)
        #print('------------------------')
        if self.state1[self.start] == 0:  # 但没车
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
        else:
            heapq.heappop(self.scheduler)
            self.addEvent(-1)
            #self.setPerformance()
            # below use self.terminal to represent the target
            self.terminal = random.choices(self.areas, weights=Pij[self.start], k=1)[0]
            self.state1[self.start] -= 1
            self.state2[self.start][self.terminal] += 1 
            self.addEvent(1)

    def stepForward(self):
        event = self.scheduler[0]
        #print(event)
        self.T, self.kind, self.start, self.terminal = event[0], event[1], event[2], event[3]
        '''
        kind of events:
        -1: customer ride a bike away
         1: a bike arrives at any area
         2: BP greater than B_
         3: a bike is fixed
         4: DP greater than D_
        '''
        if self.kind == 1: 
            self.bikeArr() # 顾客骑行到达
        elif self.kind == 2:
            self.BPover() # 坏车达到阈值
        elif self.kind == 3:
            self.repair() # 修好一辆车
        elif self.kind == 4:
            self.DPover() # 再分配
        else:# 顾客到达
            self.cusArr() #顾客到达

        return self.state1, self.state2, self.T



env = Model()
%time env.simulate()

# for r in tqdm(range(1000, 1001)):
#     for i in range(1000, 1001):
#         env.R['lambda0']=env.R['lambda1']=env.R['lambda2']=env.R['lambda3'] = r
#         env.N = i
#         result.append([r, i, env.simulate(IF_WRITEFILE)])


# test the influence of certain parameter
# for i in tqdm(range(1,100)):
#     env.N = i
#     result.append(env.simulate(IF_WRITEFILE))
# plt.plot(result)
# plt.show()

CPU times: user 4min 44s, sys: 3.94 s, total: 4min 48s
Wall time: 4min 54s


0.0

In [44]:
names=['epi', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 't']
r = pd.read_csv(FileAdd, names=names)
r[['a','b']] = r[['a','b']].astype(int)
#r.t = r.t/(RunTime+WarmTime)
r.head()

,epi,a,b,c,d,e,f,g,h,i,t
0,0,2,2,0,0,0,0,0,0,0,3.845590e-06
1,0,2,1,0,0,0,1,0,0,0,1.256588e-06
2,0,2,0,0,0,0,2,0,0,0,5.593862e-07
3,0,1,0,1,0,0,2,0,0,0,1.936154e-07
4,0,0,0,1,1,0,2,0,0,0,2.807333e-06


In [45]:
r.tail()

,epi,a,b,c,d,e,f,g,h,i,t
38675840,99,1,1,0,0,0,0,0,2,0,0.000002
38675841,99,0,1,0,1,0,0,0,2,0,0.000002
38675842,99,0,1,0,1,0,0,0,2,0,0.000002
38675843,99,0,0,0,1,1,0,0,2,0,0.000002
38675844,99,0,0,0,1,1,0,0,2,0,0.000004


In [53]:
re = []
for i in range(50):
    re.append(r[r.epi == i].drop(columns=['epi']).groupby(by=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']).sum())
re[5].t.sum()

1.0000029426798294

In [68]:
%store -r cenState
%store -r cenPorState
State, portionState = cenState, cenPorState

In [59]:
def getInterval(lst,s):
    mu, sigma = np.average(lst), np.var(lst)**0.5
    left95, right95 = mu-1.96*sigma, mu+1.96*sigma
    between = left95 <= portionState[s] <= right95
    if not between: print([s, mu, sigma, left95, right95, portionState[s], between])
    return [mu, sigma, left95, right95, portionState[s], between]

rateDict = {}
for s in State:
    series = []
    for i in range(50):
        # print(i,s)
        if s in re[i].index:
            series.append(re[i].loc[s][0])
        else: continue #series.append(series[-1])
    rateDict[s] = getInterval(series,s)
#rateDict

In [ ]:
print(np.average(rateDict[(0, 0, 4, 0, 0, 0, 0, 0, 0)]))
plt.plot(rateDict[(0, 0, 4, 0, 0, 0, 0, 0, 0)])

In [25]:
r = pd.read_csv(FileAdd,header = None)

r[(r.index<=80)&(r.index>=60)]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
60,0.0,2.0,0,0,0,1,1,0,0,4.347103,-1,0,0,4.199329,[]
61,0.0,2.0,0,0,0,1,1,0,0,4.394023,-1,1,1,4.347103,[]
62,0.0,1.0,0,0,0,1,1,1,0,4.395580,-1,1,1,4.394023,[]
63,0.0,0.0,0,0,0,1,1,2,0,4.490974,-1,0,0,4.395580,[]
64,0.0,0.0,0,0,0,1,1,2,0,4.495842,1,0,1,4.490974,[]
65,0.0,1.0,0,0,0,1,0,2,0,4.555978,-1,1,1,4.495842,[]
66,0.0,0.0,0,0,0,1,0,3,0,4.558624,-1,0,0,4.555978,[]
67,0.0,0.0,0,0,0,1,0,3,0,4.706851,1,1,0,4.558624,[]
68,1.0,0.0,0,0,0,1,0,2,0,4.771952,1,0,0,4.706851,[]
69,2.0,0.0,0,0,0,0,0,2,0,4.809415,-1,0,0,4.771952,[]


In [119]:
names=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 't']
r = pd.read_csv(FileAdd, names=names)

r['sep'] = r.t.diff()
r = r[1:]
r[['a','b']] = r[['a','b']].astype(int)
r = r.drop(columns=['t'])
#r.head()
t = r.groupby(by=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']).sum()

t.sep = t.sep/RunTime
t.sort_values('sep', ascending=False)
t.head()

sep
a b c d e f g h i          
0 0 0 0 0 0 0 0 4  0.000118
              1 3  0.000547
              2 2  0.000874
              3 1  0.000775
              4 0  0.000247